# Load data into sqlite
---
#### Note: by default, this will pull data directly from the source. Source dataset formatting can change at any time. If you encounter errors elsewhere in this pipeline, use backup data instead by setting `use_backup=True` below
(edit: `use_backup` defaults to True)

- With the Socrata ID and desired key name for all datasets:
  - Format a request url with the ID
  - Use `pd.read_csv()` to download the dataset into a dataframe.
  - Insert the table into sql, using dictionary key as table name

In [2]:
%run 1-workspace_setup.ipynb

In [2]:
use_backup = True

In [3]:
dataset_info = {
    'districts': 'mm2p-ag5w',
    'counties': 'ahgn-r8s5',
    'census_counties_2012': 'f4n4-vnyx',
    'census_counties_2013': 'm3j7-raj9',
    'census_counties_2014': 'wshk-29g7',
    'census_counties_2015': 't48m-528x',
    'census_counties_2016': 'eghh-ua8y',
    'census_counties_2017': 'ewkj-ipn7',
    'census_counties_2018': 'xum2-smvh',
    'census_counties_2019': '8j3i-rjn4',
    'census_counties_field_desc': 'qten-sdpn',
    'county_population': 'eeah-cmy8',
    'crime_16_19': 'j6g4-gayk',
    'crime_97_15': '6vnq-az4b',
    'dist_grad_rate': 'cfyh-6xxg',
    'dist_mobility_demographics': 'rg84-k4d3',
    'dist_student_mobility': '6wcd-ysh5',
}

In [3]:
format_url = lambda id: f'https://data.colorado.gov/api/views/{id}/rows.csv?accessType=DOWNLOAD'
backup_file_fmt = lambda name: f"backup/{name}.parquet.gzip"


def query(txt: str, con: str) -> list:
    cur = sql[con]['con']().cursor()
    return cur.execute(txt).fetchall()


def table_exists(con: str, name):
    c = sql[con]['con']().cursor()
                
    # get the count of tables with the name
    c.execute(f"""
    SELECT
        count(name)
    FROM sqlite_master
    WHERE type='table'
        AND name='{name}'
    """)

    # if the count is 1, then table exists
    if c.fetchone()[0] == 1:
        return True
    return False

## Load tables and write to raw

In [5]:
for name, id in dataset_info.items():
    if table_exists('raw', name):
        continue
    if use_backup == True:
        df = pd.read_parquet(backup_file_fmt(name))
    else:
        df = pd.read_csv(format_url(id))
    write_raw(df, name)

## See tables in raw and main

In [6]:
raw_tables = query('''SELECT name FROM sqlite_master WHERE type='table';''', con='raw')
main_tables = query('''SELECT name FROM sqlite_master WHERE type='table';''', con='main')

print("RAW:")
print(*[t[0] for t in raw_tables], sep='\n')
print("\nMAIN:")
print(*[t[0] for t in main_tables], sep='\n')

RAW:
districts
counties
census_counties_2012
census_counties_2013
census_counties_2014
census_counties_2015
census_counties_2016
census_counties_2017
census_counties_2018
census_counties_2019
census_counties_field_desc
county_population
crime_16_19
crime_97_15
dist_grad_rate
dist_mobility_demographics
dist_student_mobility

MAIN:



### Save to backup?

In [7]:
def save_raw_data_to_backup():
    """
    No need to use this unless you want to backup current
    state of raw datasets as parquet files
    """
    for name in dataset_info.keys():
        df = read_raw(name)
        df.to_parquet(backup_file_fmt(name), compression='gzip', index=False)

# save_raw_data_to_backup()

### Clear databases

In [ ]:
def empty_raw():
    raw_tables = query("SELECT name FROM sqlite_master WHERE type='table'", con='raw')
    con = get_con_raw()
    cur = con.cursor()
    for name in raw_tables:
        print(name[0])
        cur.execute(f"DROP TABLE IF EXISTS {name[0]}")
    con.commit()
    cur.execute("VACUUM;")

# empty_raw()

In [ ]:
def empty_main():
    main_tables = query("SELECT name FROM sqlite_master WHERE type='table'", con='main')
    con = get_con_main()
    cur = con.cursor()
    for name in main_tables:
        print(name[0])
        cur.execute(f"DROP TABLE IF EXISTS {name[0]}")
    con.commit()
    cur.execute("VACUUM;")

# empty_main()